In [2]:
!pip install -U matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.0
    Uninstalling matplotlib-3.8.0:
      Successfully uninstalled matplotlib-3.8.0


In [3]:
%reload_ext autoreload
%autoreload 2
import os
import pandas as pd
import sys
from sklearn.utils import shuffle
# add the 'src' directory as one where we can import modules
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Increase figure resolution
%config InlineBackend.figure_format = 'retina'
sns.set()

load the data


In [4]:
from google.colab import drive
drive.mount('/content/drive')

data_frame = pd.read_csv('/content/drive/MyDrive/EEG/features_raw.csv')

data_frame.shape



Mounted at /content/drive


(8064, 33)

In [5]:
data_frame = pd.read_csv('/content/drive/MyDrive/EEG/features_raw.csv')
data_frame.head(5)

,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,Cz,C4,T8,CP6,CP2,P4,P8,PO4,O2,class
0,0.057813,-1.335266,4.640480,0.219573,7.473817,2.314842,1.918097,-9.257533,9.089943,-7.104519,...,-2.241480,1.415335,2.406646,12.864059,4.021099,-2.828598,-2.588735,2.637905,-5.226618,abnormal
1,1.367408,10.259654,3.345409,7.897852,-2.446051,-1.655035,-6.301423,-7.290317,-3.546453,-5.705187,...,-2.568397,-5.651418,-0.096730,-4.930759,-1.722504,-6.111309,0.094893,-3.521353,1.887093,normal
2,-1.783132,4.133553,-0.951680,-1.624803,-1.827309,-2.280364,-2.279225,9.151344,-0.239575,-0.057604,...,-2.132823,-0.521117,8.605298,-4.499946,-3.232839,-4.249645,-3.687167,-7.383004,-4.489537,abnormal
3,-3.690217,-0.814000,2.295469,0.901445,8.323679,1.127906,6.356886,11.642082,9.354154,-1.662478,...,-0.506117,-1.154866,-3.940251,7.390881,2.129897,-0.794675,-1.959021,2.774530,-6.323060,abnormal
4,2.137114,6.420466,6.122230,10.015321,3.106394,3.183129,3.658535,4.571793,4.917712,-2.325940,...,1.813907,-6.444635,-27.680880,0.641364,1.996658,-0.445779,2.614021,6.161845,3.308816,normal


In [6]:
# Check for NaN or null values in the entire DataFrame
print("Total null values in the DataFrame:")
print(data_frame.isnull().sum())

# Check for any empty strings
print("\nColumns with empty string values:")
empty_string_columns = data_frame.apply(lambda col: (col == '').sum())
print(empty_string_columns[empty_string_columns > 0])

# Check for columns with zero values
print("\nColumns with zero values:")
zero_value_columns = (data_frame == 0).sum()
print(zero_value_columns[zero_value_columns > 0])

# Check for infinite values
print("\nColumns with infinite values:")
inf_columns = data_frame.isin([np.inf, -np.inf]).sum()
print(inf_columns[inf_columns > 0])

# Comprehensive null and empty value check
null_summary = pd.DataFrame({
   'Null Values': data_frame.isnull().sum(),
   'Empty Strings': data_frame.apply(lambda col: (col == '').sum()),
   'Zero Values': (data_frame == 0).sum()
})
print("\nComprehensive Null and Empty Value Summary:")
print(null_summary[null_summary.sum(axis=1) > 0])

Total null values in the DataFrame:
Fp1      0
AF3      0
F3       0
F7       0
FC5      0
FC1      0
C3       0
T7       0
CP5      0
CP1      0
P3       0
P7       0
PO3      0
O1       0
Oz       0
Pz       0
Fp2      0
AF4      0
Fz       0
F4       0
F8       0
FC6      0
FC2      0
Cz       0
C4       0
T8       0
CP6      0
CP2      0
P4       0
P8       0
PO4      0
O2       0
class    0
dtype: int64

Columns with empty string values:
Series([], dtype: int64)

Columns with zero values:
Series([], dtype: int64)

Columns with infinite values:
Series([], dtype: int64)

Comprehensive Null and Empty Value Summary:
Empty DataFrame
Columns: [Null Values, Empty Strings, Zero Values]
Index: []


**Data Normalization**

Normalize the EEG data to have zero mean and unit variance. This ensures that all channels contribute equally to the model.

In [7]:
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = data_frame.iloc[:, :-1]  # EEG channels
y = data_frame['class']  # Target class

# Normalize EEG channel data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Reintegrate into a DataFrame for easier manipulation
X_normalized = pd.DataFrame(X_normalized, columns=X.columns)

# Display head of the normalized data
print("Head of Normalized Data:")
display(X_normalized.head())


Head of Normalized Data:


,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,FC2,Cz,C4,T8,CP6,CP2,P4,P8,PO4,O2
0,0.020430,-0.072700,0.937690,0.010626,0.503970,0.444277,0.139075,-0.456731,0.373940,-0.582007,...,-0.353806,-0.570612,0.121918,0.098787,0.350460,0.185935,-0.212269,-0.217756,0.074962,-0.556270
1,0.324732,0.535976,0.676033,0.403802,-0.158530,-0.315721,-0.441211,-0.360514,-0.138932,-0.468153,...,-0.019613,-0.653016,-0.489083,-0.007921,-0.128971,-0.075536,-0.459485,0.000724,-0.090594,0.204507
2,-0.407338,0.214386,-0.192154,-0.083818,-0.117207,-0.435435,-0.157250,0.443647,-0.004716,-0.008649,...,0.125473,-0.543223,-0.045510,0.363010,-0.117364,-0.144292,-0.319285,-0.307182,-0.194393,-0.477443
3,-0.850475,-0.045336,0.463903,0.045542,0.560728,0.217048,0.452447,0.565469,0.384663,-0.139226,...,-0.172862,-0.133189,-0.100305,-0.171755,0.203000,0.099840,-0.059097,-0.166490,0.078634,-0.673529
4,0.503584,0.334438,1.237064,0.512230,0.212291,0.610503,0.261948,0.219661,0.204602,-0.193208,...,-0.162521,0.451605,-0.557666,-1.183719,0.021154,0.093775,-0.032822,0.205813,0.169683,0.356553


Bandpass Filtering

Filter EEG signals to retain relevant frequencies (e.g., 0.5–50 Hz).

In [8]:
from scipy.signal import butter, filtfilt

# Define a bandpass filter
def bandpass_filter(data, lowcut=0.5, highcut=50, fs=128, order=4):
    nyquist = 0.5 * fs  # Nyquist Frequency
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Apply to each channel
filtered_signals = X.apply(lambda col: bandpass_filter(col.values), axis=0)

# Convert to DataFrame and display head
filtered_signals = pd.DataFrame(filtered_signals, columns=X.columns)
print("Head of Filtered Data:")
display(filtered_signals.head())


Head of Filtered Data:


,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,FC2,Cz,C4,T8,CP6,CP2,P4,P8,PO4,O2
0,-0.702821,-1.006378,0.299734,-1.104233,-1.109473,0.289878,-0.591591,2.279865,-0.206921,1.549939,...,4.996777,1.791152,1.047053,-0.687985,-0.752966,-0.073036,-0.256813,-0.751578,-2.111758,-0.649800
1,0.739386,10.829968,-0.941706,6.799423,-11.058134,-3.694647,-8.926208,4.659694,-12.943449,2.819895,...,34.343051,1.341214,-6.175258,-3.343249,-18.755134,-5.931755,-3.675967,1.936573,-8.395807,6.506087
2,-2.690842,4.070546,-5.147305,-3.239637,-9.844068,-4.174919,-4.438261,19.321401,-8.823488,8.268247,...,44.736298,1.800927,-0.633410,6.019596,-16.981377,-6.918325,-1.601008,-1.966066,-11.612089,-0.241029
3,-4.214390,-0.057176,-1.842037,0.079477,-0.062011,-0.880393,3.727792,23.473038,0.097971,6.537803,...,19.987394,3.218762,-1.834445,-7.396388,-6.233441,-2.036547,1.506529,-0.071653,-1.989405,-1.723984
4,1.333904,6.480982,2.047679,8.512646,-4.735789,1.328648,1.496336,14.770601,-3.493340,5.719280,...,18.687297,5.555385,-6.615505,-30.057031,-11.640812,-1.677866,2.056095,4.228426,1.958711,7.437218


Downsampling


Reduce the data's dimensionality by keeping fewer samples (e.g., reducing sampling frequency)

In [9]:
downsample_factor = 2  # Adjust as needed
X_downsampled = filtered_signals.iloc[::downsample_factor, :]

# Display head of the downsampled data
print("Head of Downsampled Data:")
display(X_downsampled.head())


Head of Downsampled Data:


,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,FC2,Cz,C4,T8,CP6,CP2,P4,P8,PO4,O2
0,-0.702821,-1.006378,0.299734,-1.104233,-1.109473,0.289878,-0.591591,2.279865,-0.206921,1.549939,...,4.996777,1.791152,1.047053,-0.687985,-0.752966,-0.073036,-0.256813,-0.751578,-2.111758,-0.649800
2,-2.690842,4.070546,-5.147305,-3.239637,-9.844068,-4.174919,-4.438261,19.321401,-8.823488,8.268247,...,44.736298,1.800927,-0.633410,6.019596,-16.981377,-6.918325,-1.601008,-1.966066,-11.612089,-0.241029
4,1.333904,6.480982,2.047679,8.512646,-4.735789,1.328648,1.496336,14.770601,-3.493340,5.719280,...,18.687297,5.555385,-6.615505,-30.057031,-11.640812,-1.677866,2.056095,4.228426,1.958711,7.437218
6,-3.419226,-10.957381,-8.808890,-13.613288,-7.436584,-1.646218,1.265454,-4.047264,1.132590,11.288300,...,22.226145,4.506946,10.960488,8.183505,2.785414,2.258154,9.162569,-0.238732,3.121824,4.778681
8,1.802832,-1.954518,-7.356001,-3.398053,-13.870178,-7.337530,-8.023966,3.146303,-16.854522,7.801847,...,37.435597,2.315802,1.432335,-7.132322,-12.975232,-2.256849,9.092382,4.280947,1.886460,13.907283


Combine Features

Merge time-domain and frequency-domain features into a single dataset.

In [10]:
# Compute time-domain features such as mean, variance, and standard deviation
time_features = X_downsampled.apply(lambda col: {
    'mean': col.mean(),
    'variance': col.var(),
    'std_dev': col.std()
}, axis=0).T

# Convert to DataFrame and display head
time_features_df = pd.DataFrame(time_features)
print("Head of Time-Domain Features:")
display(time_features_df.head())


Head of Time-Domain Features:


,0
Fp1,"{'mean': -0.013397017392424574, 'variance': 18..."
AF3,"{'mean': 0.01261490969550654, 'variance': 362...."
F3,"{'mean': 0.023850567313397915, 'variance': 25...."
F7,"{'mean': 0.09261833986791906, 'variance': 385...."
FC5,"{'mean': 0.04860670376516377, 'variance': 226...."


In [11]:
from scipy.fft import fft
import numpy as np

def compute_fft_features(signal, fs=128):
    fft_values = fft(signal)
    psd = np.abs(fft_values) ** 2
    freqs = np.fft.fftfreq(len(psd), 1/fs)
    return {
        'delta_power': psd[(freqs >= 0.5) & (freqs < 4)].mean(),
        'theta_power': psd[(freqs >= 4) & (freqs < 8)].mean(),
        'alpha_power': psd[(freqs >= 8) & (freqs < 13)].mean(),
        'beta_power': psd[(freqs >= 13) & (freqs < 30)].mean(),
        'gamma_power': psd[(freqs >= 30) & (freqs <= 50)].mean()
    }

# Apply FFT feature extraction to each channel
fft_features = X_downsampled.apply(lambda col: compute_fft_features(col.values), axis=0)

# Convert to DataFrame and display head
fft_features_df = pd.DataFrame(fft_features)
print("Head of Frequency-Domain Features:")
display(fft_features_df.head())


Head of Frequency-Domain Features:


,0
Fp1,"{'delta_power': 4577.43800936555, 'theta_power..."
AF3,"{'delta_power': 6621.469291831936, 'theta_powe..."
F3,"{'delta_power': 52117.281744967106, 'theta_pow..."
F7,"{'delta_power': 296646.4722753416, 'theta_powe..."
FC5,"{'delta_power': 187344.95866999635, 'theta_pow..."


In [12]:
# Ensure that X_downsampled, time_features_df, and fft_features_df have aligned indexes
X_downsampled = X_downsampled.reset_index(drop=True)
time_features_df = time_features_df.reset_index(drop=True)
fft_features_df = fft_features_df.reset_index(drop=True)

# Now combine the features
feature_data = pd.concat([X_downsampled, time_features_df, fft_features_df], axis=1)

# Display the head of the combined feature data to check alignment
print("Head of Combined Feature Data:")
display(feature_data.head())


Head of Combined Feature Data:


,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,C4,T8,CP6,CP2,P4,P8,PO4,O2,0,0
0,-0.702821,-1.006378,0.299734,-1.104233,-1.109473,0.289878,-0.591591,2.279865,-0.206921,1.549939,...,1.047053,-0.687985,-0.752966,-0.073036,-0.256813,-0.751578,-2.111758,-0.649800,"{'mean': -0.013397017392424574, 'variance': 18...","{'delta_power': 4577.43800936555, 'theta_power..."
1,-2.690842,4.070546,-5.147305,-3.239637,-9.844068,-4.174919,-4.438261,19.321401,-8.823488,8.268247,...,-0.633410,6.019596,-16.981377,-6.918325,-1.601008,-1.966066,-11.612089,-0.241029,"{'mean': 0.01261490969550654, 'variance': 362....","{'delta_power': 6621.469291831936, 'theta_powe..."
2,1.333904,6.480982,2.047679,8.512646,-4.735789,1.328648,1.496336,14.770601,-3.493340,5.719280,...,-6.615505,-30.057031,-11.640812,-1.677866,2.056095,4.228426,1.958711,7.437218,"{'mean': 0.023850567313397915, 'variance': 25....","{'delta_power': 52117.281744967106, 'theta_pow..."
3,-3.419226,-10.957381,-8.808890,-13.613288,-7.436584,-1.646218,1.265454,-4.047264,1.132590,11.288300,...,10.960488,8.183505,2.785414,2.258154,9.162569,-0.238732,3.121824,4.778681,"{'mean': 0.09261833986791906, 'variance': 385....","{'delta_power': 296646.4722753416, 'theta_powe..."
4,1.802832,-1.954518,-7.356001,-3.398053,-13.870178,-7.337530,-8.023966,3.146303,-16.854522,7.801847,...,1.432335,-7.132322,-12.975232,-2.256849,9.092382,4.280947,1.886460,13.907283,"{'mean': 0.04860670376516377, 'variance': 226....","{'delta_power': 187344.95866999635, 'theta_pow..."


In [20]:
!pip install catboost
!pip install dask[dataframe]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.9 MB/s eta 0:00:00


**Training with 3 models**

In [21]:

# Encode the target labels if they are categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.5, random_state=42, stratify=y_encoded)

# Standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Import additional models
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Initialize models
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(random_state=42, verbose=0),
    "Support Vector Machine (Linear)": SVC(kernel='linear', random_state=42),
    "Support Vector Machine (RBF)": SVC(kernel='rbf', random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Extra Trees": ExtraTreesClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "Gaussian Naive Bayes": GaussianNB()
}

# Train and evaluate each model
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    roc_auc_score,
    f1_score
)

# Optional: Cross-validation for more robust evaluation
from sklearn.model_selection import cross_val_score

for model_name, model in models.items():
    print(f"Training and evaluating: {model_name}")

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nAccuracy Score:")
    print(accuracy_score(y_test, y_pred))

    print("\nROC AUC Score:")
    try:
        print(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))
    except:
        print("ROC AUC Score not available for this model")

    print("\nCross-Validation Scores:")
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

    print("-" * 50)

Training and evaluating: Random Forest

Classification Report:
              precision    recall  f1-score   support

    abnormal       1.00      1.00      1.00      2043
      normal       1.00      1.00      1.00      1989

    accuracy                           1.00      4032
   macro avg       1.00      1.00      1.00      4032
weighted avg       1.00      1.00      1.00      4032


Confusion Matrix:
[[2043    0]
 [   0 1989]]

Accuracy Score:
1.0

ROC AUC Score:
1.0

Cross-Validation Scores:
Mean CV Score: 0.9990 (+/- 0.0010)
--------------------------------------------------
Training and evaluating: Gradient Boosting

Classification Report:
              precision    recall  f1-score   support

    abnormal       1.00      1.00      1.00      2043
      normal       1.00      1.00      1.00      1989

    accuracy                           1.00      4032
   macro avg       1.00      1.00      1.00      4032
weighted avg       1.00      1.00      1.00      4032


Confusion Matrix

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:59:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:59:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Classification Report:
              precision    recall  f1-score   support

    abnormal       1.00      1.00      1.00      2043
      normal       1.00      1.00      1.00      1989

    accuracy                           1.00      4032
   macro avg       1.00      1.00      1.00      4032
weighted avg       1.00      1.00      1.00      4032


Confusion Matrix:
[[2043    0]
 [   4 1985]]

Accuracy Score:
0.9990079365079365

ROC AUC Score:
0.9999985234501949

Cross-Validation Scores:


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:59:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:59:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:59:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:59:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mean CV Score: 0.9983 (+/- 0.0012)
--------------------------------------------------
Training and evaluating: LightGBM
[LightGBM] [Info] Number of positive: 1990, number of negative: 2042
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002009 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Number of data points in the train set: 4032, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493552 -> initscore=-0.025795
[LightGBM] [Info] Start training from score -0.025795
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1


Classification Report:
              precision    recall  f1-score   support

    abnormal       0.99      0.99      0.99      2043
      normal       0.99      0.99      0.99      1989

    accuracy                           0.99      4032
   macro avg       0.99      0.99      0.99      4032
weighted avg       0.99      0.99      0.99      4032


Confusion Matrix:
[[2020   23]
 [  25 1964]]

Accuracy Score:
0.9880952380952381

ROC AUC Score:
0.9992627094639706

Cross-Validation Scores:
Mean CV Score: 0.9893 (+/- 0.0078)
--------------------------------------------------
Training and evaluating: Gaussian Naive Bayes

Classification Report:
              precision    recall  f1-score   support

    abnormal       0.82      0.80      0.81      2043
      normal       0.80      0.82      0.81      1989

    accuracy                           0.81      4032
   macro avg       0.81      0.81      0.81      4032
weighted avg       0.81      0.81      0.81      4032


Confusion Matrix:
[[16